<a href="https://colab.research.google.com/github/look4pritam/KnowledgeDistillation/blob/master/Notebooks/KnowledgeDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 47.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import pytorch_lightning as pl

# Teacher Model.

In [3]:
class TeacherCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))  # 32x32 -> 16x16
        x = F.relu(F.max_pool2d(self.conv2(x), 2))  # 16x16 -> 8x8
        x = F.relu(F.max_pool2d(self.conv3(x), 2))  # 8x8 -> 4x4
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Lightning Module for Teacher Training.

In [4]:
class TeacherLightning(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss_teacher", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss_teacher", loss, prog_bar=True)
        self.log("val_acc_teacher", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("test_loss_teacher", loss)
        self.log("test_acc_teacher", acc)
        return acc

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

# Student Model.

In [5]:
class StudentModel(pl.LightningModule):
    def __init__(self, teacher_model, temperature=3, alpha=0.5, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha
        self.lr = lr

        # Freeze teacher parameters
        self.teacher.eval()
        for param in self.teacher.parameters():
            param.requires_grad = False

        # Student architecture
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def setup(self, stage=None):
        self.teacher.to(self.device)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch

        # Get teacher predictions
        with torch.no_grad():
            teacher_logits = self.teacher(x)

        student_logits = self(x)

        # Loss components
        ce_loss = F.cross_entropy(student_logits, y)
        student_logits_dist = student_logits / self.temperature
        teacher_logits_dist = teacher_logits / self.temperature

        # KL Divergence
        student_log_prob = F.log_softmax(student_logits_dist, dim=1)
        teacher_prob = F.softmax(teacher_logits_dist, dim=1)
        kl_loss = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

        # Total loss
        distillation_loss = (self.temperature ** 2) * kl_loss
        loss = (1 - self.alpha) * ce_loss + self.alpha * distillation_loss

        # Logging
        self.log("train_loss_student", loss)
        self.log("ce_loss", ce_loss)
        self.log("dist_loss", distillation_loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        student_logits = self(x)
        loss = F.cross_entropy(student_logits, y)
        preds = torch.argmax(student_logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss_student", loss, prog_bar=True)
        self.log("val_acc_student", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        student_logits = self(x)
        loss = F.cross_entropy(student_logits, y)
        preds = torch.argmax(student_logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("test_loss_student", loss)
        self.log("test_acc_student", acc)
        return acc

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# CIFAR-10 Data Module.

In [6]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465),
                                 (0.2023, 0.1994, 0.2010))
        ])

    def prepare_data(self):
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = datasets.CIFAR10(root='./data', train=True,
                                                 transform=self.transform)
            self.val_dataset = datasets.CIFAR10(root='./data', train=False,
                                               transform=self.transform)
        if stage == 'test':
            self.test_dataset = datasets.CIFAR10(root='./data', train=False,
                                                transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

# Train Teacher Model.

In [7]:
teacher_model = TeacherCNN()
teacher_lightning = TeacherLightning(teacher_model, learning_rate=1e-3)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [8]:
dm = CIFAR10DataModule(batch_size=32)

In [9]:
trainer_teacher = pl.Trainer(
      max_epochs=20,
      accelerator="auto",
      devices=1 if torch.cuda.is_available() else None,
     logger=pl.loggers.TensorBoardLogger("logs/teacher")
    )
trainer_teacher.fit(teacher_lightning, dm)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
100%|██████████| 170M/170M [00:18<00:00, 9.14MB/s]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | TeacherCNN       | 1.1 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.590     Total estimated model params size (MB)
7    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


# Save trained teacher model.

In [10]:
torch.save(teacher_model.state_dict(), "teacher_model.pth")

# Train Student Model using the trained Teacher Model.

In [11]:
teacher = TeacherCNN()
teacher.load_state_dict(torch.load("teacher_model.pth"))
teacher.eval()

TeacherCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [12]:
student = StudentModel(teacher, temperature=3, alpha=0.5, lr=1e-3)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'teacher_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['teacher_model'])`.


In [13]:
trainer_student = pl.Trainer(
        max_epochs=10,
        accelerator="auto",
        devices=1 if torch.cuda.is_available() else None,
        logger=pl.loggers.TensorBoardLogger("logs/student")
)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
trainer_student.fit(student, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | teacher | TeacherCNN | 1.1 M  | eval 
1 | conv1   | Conv2d     | 448    | train
2 | conv2   | Conv2d     | 4.6 K  | train
3 | fc1     | Linear     | 262 K  | train
4 | fc2     | Linear     | 1.3 K  | train
-----------------------------------------------
268 K     Trainable params
1.1 M     Non-trainable params
1.4 M     Total params
5.664     Total estimated model params size (MB)
4         Modules in train mode
6         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
